In [13]:
import os
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.postprocess import Simulation
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
import scipy.constants as consts
import scipy.integrate as intg
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Introduction

This notebook is intended to check the consistency between 3 different methods for calculating the absorption of a layer in S4.
The methods are as follows

1. Using the GetPowerFlux method and taking the difference between what enters the top and leaves the bottom. 
2. Using the GetVolumeIntegral function
3. Integrate the raw field output numerically

All of these methods should agree with one another, so let's verify this. Below I just set up the sim so it's ready for all subsequent computations

In [2]:
conf = Config('GaAs_slab.yml')
sim = Simulator(conf)
sim = setup_sim(sim)

In [3]:
fluxes = sim.get_fluxes()

In [4]:
print(sim.flux_dict)

{'Substrate': ((1.7579049646056364+0.011467893212008501j), (-0.0987989801440658-0.011467893212008501j)), 'Below_Air': ((0.47642383837444036+0j), 0j), 'Air': ((2+0j), (-0.3408940155385014+0j)), 'Substrate_bottom': ((0.7123928372666464+0.011930675216029193j), (-0.23596899889220918-0.011930675216029193j)), 'Below_Air_bottom': ((0.47642383837444036+0j), 0j), 'Air_bottom': ((2+0j), (-0.3408940155385014+0j))}


# Poynting Flux Method

In [5]:
summed_absorbed_power = 0
abs_dict_fluxmethod = {}
for layer, (forw, back) in fluxes.items():
    if '_bottom' in layer:
        continue
    incident_power = np.absolute(forw)
    reflected_power = np.absolute(back)
    print('-'*25)
    print('Layer: {}'.format(layer))
    print('Incident Power: {}'.format(incident_power))
    print('Reflected Power: {}'.format(reflected_power))
    bottom = layer+'_bottom'
    transmitted_power = np.absolute(fluxes[bottom][0])
    bottom_reflected_power = np.absolute(fluxes[bottom][1])
    print('Transmitted Power: {}'.format(transmitted_power))
    print('Backward_bottom: {}'.format(bottom_reflected_power))
    #absorbed = incident_power + bottom_reflected_power - transmitted_power - reflected_power
    flux_top = np.absolute(forw+back)
    flux_bottom = np.absolute(fluxes[bottom][0]+fluxes[bottom][1]) 
    absorbed = flux_top - flux_bottom
    abs_dict_fluxmethod[layer] = absorbed
    print('Absorbed in Layer: {}'.format(absorbed))
    summed_absorbed_power += absorbed
#print('Summed Absorption= {}'.format(np.absolute(summed_absorbed_power)))
print('-'*25)
print('Summed Absorption= {}'.format(summed_absorbed_power))
print('GaAs Absorption = {}'.format(.5*abs_dict_fluxmethod['Substrate']))

-------------------------
Layer: Substrate
Incident Power: 1.7579423702612853
Reflected Power: 0.09946230970689117
Transmitted Power: 0.712492733717287
Backward_bottom: 0.23627041594178014
Absorbed in Layer: 1.1826821460871333
-------------------------
Layer: Below_Air
Incident Power: 0.47642383837444036
Reflected Power: 0.0
Transmitted Power: 0.47642383837444036
Backward_bottom: 0.0
Absorbed in Layer: 0.0
-------------------------
Layer: Air
Incident Power: 2.0
Reflected Power: 0.3408940155385014
Transmitted Power: 2.0
Backward_bottom: 0.3408940155385014
Absorbed in Layer: 0.0
-------------------------
Summed Absorption= 1.1826821460871333
GaAs Absorption = 0.5913410730435666


# GetVolumeIntegral Method

We use the provided S4 function to compute

\begin{equation}
    \int \epsilon | \vec{E} | ^2 dV
\end{equation}
  
where the integral is over the volume of the layer (that is what `Quantity = "E"` means). The total absorbed power is 

\begin{equation}
    P_{abs} = \frac{\omega}{2} \int Im(\epsilon) | \vec{E} | ^2 dV
\end{equation}

Where here we need to convert $\omega$ into the Lorentz-Heaviside units used by S4. $|E|^2$ should be a purely real quantity,
meaning we can take the imaginary part of the result of the integration and multiply by $\omega / 2$ to get the absorbed 
power.

In [14]:
int_result = sim.s4.GetLayerVolumeIntegral(Layer="Substrate", Quantity="E")
int_result2 = sim.s4.GetLayerVolumeIntegral(Layer="Substrate", Quantity="e")
epsilon = sim._get_epsilon(os.path.expandvars(
    '$HOME/software/nanowire/nanowire/NK/006_GaAs_nk_Walker_modified_Hz.txt'))
print('Epsilon = {}'.format(epsilon))
si_freq = sim.conf[('Simulation', 'params', 'frequency', 'value')]
base_unit = sim.conf[('Simulation', 'base_unit')]
c_conv = consts.c / base_unit 
f_conv = si_freq / c_conv
print('Converted freq = {}'.format(f_conv))
print("Pabs = {}".format(.25*f_conv*int_result))
print("Pabs2  = {}".format(complex(.25*f_conv*epsilon.real*int_result2.real,.25*f_conv*epsilon.imag*int_result2.imag)))

Epsilon = (13.767280000000001+0.8017920000000001j)
Converted freq = 1.3342563807926082
Pabs = (0.809375168317913-7.507962969271358e-18j)
Pabs2  = (0.8066392288407569-0.00273593947715617j)


So this is a bit odd. Two methods of computing the same exact integral do not agree about the imaginary componented. The only 
difference here is I pulled the $\epsilon$ outside the integral because we know it is uniform over an 
isotropic slab. If we do the same thing but using the Lua API and the latest version of S4 on the master branch we get
similiar results. The lua script is below.

In [8]:
%cat volume_integral.lua

S = S4.NewSimulation()
-- Params
period = .25
numbasis = 300
eps_real = 13.77
eps_imag = 0.8
thickness = .5
freq = 1.3342563807926082
--
S:SetVerbosity(9)
S:SetLattice({period,0}, {0,period})
S:SetNumG(numbasis)
S:AddMaterial('Vacuum', {1.0, 0})
S:AddMaterial('GaAs', {eps_real, eps_imag})
S:AddLayer('top', 0, 'Vacuum')
S:AddLayer('slab', thickness, 'GaAs')
S:AddLayerCopy('bottom', 0, 'top')
S:SetExcitationPlanewave(
        {0, 0}, -- phi in [0,180), theta in [0,360)
        {1, 0}, -- s-polarization amplitude and phase in degrees
        {0, 0}) -- p-polarization
S:SetFrequency(freq)
tfr, tbr, tfi, tbi = S:GetPowerFlux('top')
bfr, bbr, bfi, bbi = S:GetPowerFlux('bottom')
print("### Top ###")
print(tfr)
print(tbr)
print(tfi)
print(tbi)
print("### Bottom ###")
print(bfr)
print(bbr)
print(bfi)
print(bbi)
pabs_r = (tfr - bfr) + (tbr- bbr)
pabs_i = (tfi - bfi) + (tbi- bbi)
print("Power Absorbed Real: " .. pabs_r)
print("Power Absorbed Imag: " .. pabs_i)
int_r, int_i = S:GetLayerElectricEne

Here is what we get if we run it

In [10]:
!!$HOME/software/S4/build/S4 volume_integral.lua

['[1] Computing solution in layer: top',
 '[1] Computing modes of layer: top',
 '[1] Computing modes of layer: slab',
 '[1] Computing solution in layer: bottom',
 '### Top ###',
 '1',
 '-0.1698995526786',
 '6.9388939039072e-18',
 '-6.9388939039072e-18',
 '### Bottom ###',
 '0.23890877416073',
 '0',
 '0',
 '0',
 'Power Absorbed Real: 0.59119167316067',
 'Power Absorbed Imag: 0',
 '[1] Computing solution in layer: slab',
 'Integral of epsilon |E|^2 Real: 0.81113679072971',
 'Integral of epsilon |E|^2 Imag: 2.9098357364357e-18',
 'Integral of |E|^2 epsilon outside Real: 0.80840817242183',
 'Integral of |E|^2 epsilon outside Imag: -0.0027286183078787']

So first of all it seems like the integrals are time-averaged when using the Lua API, but are NOT time averaged when 
using the Python API (thats why we multiply by .25 when using the Python API, but multiply by .5 using the Lua API. 
Factor of .5 from the equation for Pabs, and factor of .5 from time averaging). When doing this, the results seem to
agree quite well. However, regardless of which API we use, it is troubling that the imaginary components do not agree
between the two rearrangements of the expression for the absorbed power.

# Integrate Raw Fields

This approach collects the raw fields from S4 and integrates them over the volume of the layer

In [17]:
sim.get_field()

In [20]:
Ex = np.absolute(sim.data['Ex'])
Ey = np.absolute(sim.data['Ey'])
Ez = np.absolute(sim.data['Ez'])
normE = np.sqrt(Ex**2 + Ey**2 + Ex**2)
zsamps = sim.conf[('Simulation', 'z_samples')]
xsamps = sim.conf[('Simulation', 'x_samples')]
ysamps = sim.conf[('Simulation', 'y_samples')]
height = sim.get_height()
z_vals = np.linspace(0, height, zsamps)
x_vals = np.linspace(0, sim.period, xsamps)
y_vals = np.linspace(0, sim.period, ysamps)
z_integral = intg.trapz(normE, x=z_vals, axis=0)
x_integral = intg.trapz(z_integral, x=x_vals, axis=0)
y_integral = intg.trapz(x_integral, x=y_vals, axis=0)
epsilon = sim._get_epsilon(os.path.expandvars(
    '$HOME/software/nanowire/nanowire/NK/006_GaAs_nk_Walker_modified_Hz.txt'))
print(epsilon)
si_freq = sim.conf[('Simulation', 'params', 'frequency', 'value')]
base_unit = sim.conf[('Simulation', 'base_unit')]
c_conv = consts.c / base_unit 
f_conv = si_freq / c_conv
print("Pabs = {}".format(.5*f_conv*epsilon.imag*y_integral))

(13.767280000000001+0.8017920000000001j)
Pabs = 0.011723507855552075


In [21]:
sim_proc = Simulation(simulator=sim)
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
layer_name = 'Substrate'
layer_obj = sim_proc.layers[layer_name]
print("Layer : {}".format(layer_name))
si_freq = sim_proc.conf[('Simulation', 'params', 'frequency', 'value')]
base_unit = sim_proc.conf[('Simulation', 'base_unit')]
c_conv = consts.c / base_unit 
f_conv = si_freq / c_conv
n_mat, k_mat = layer_obj.get_nk_matrix(si_freq)
print(n_mat[0,0])
print(k_mat[0,0])
print(2*n_mat[0,0,]*k_mat[0,0])
# n and k could be functions of space, so we need to multiply the
# fields by n and k before integrating
print('Slice: {}'.format(layer_obj.slice))
arr_slice = Esq[layer_obj.slice]*n_mat*k_mat
zsamps = layer_obj.iend - layer_obj.istart
z_vals = np.linspace(0, layer_obj.thickness, zsamps)
x_vals = np.linspace(0, sim_proc.period, sim_proc.x_samples)
y_vals = np.linspace(0, sim_proc.period, sim_proc.y_samples)
z_integral = intg.trapz(arr_slice, x=z_vals, axis=0)
x_integral = intg.trapz(z_integral, x=x_vals, axis=0)
y_integral = intg.trapz(x_integral, x=y_vals, axis=0)
p_abs_imag = .5*f_conv*y_integral
arr_slice = Esq[layer_obj.slice]*(n_mat**2 - k_mat**2)
zsamps = layer_obj.iend - layer_obj.istart
z_vals = np.linspace(0, layer_obj.thickness, zsamps)
x_vals = np.linspace(0, sim_proc.period, sim_proc.x_samples)
y_vals = np.linspace(0, sim_proc.period, sim_proc.y_samples)
z_integral = intg.trapz(arr_slice, x=z_vals, axis=0)
x_integral = intg.trapz(z_integral, x=x_vals, axis=0)
y_integral = intg.trapz(x_integral, x=y_vals, axis=0)
p_abs_real = .5*f_conv*y_integral
print("Integrated Absorbed Imag: {}".format(p_abs_imag))
print("Integrated Absorbed Real: {}".format(p_abs_real))


Layer : Substrate
3.71257179904
0.108
0.801915508592
Slice: (slice(0, 150, None), Ellipsis)
Integrated Absorbed Imag: 0.0029416146665228347
Integrated Absorbed Real: 0.10103438717459336
